In [ ]:
# ========================================

TAXON = 'Canis'

# ========================================

import requests
from IPython.display import IFrame
import re


def show_site(url):
    print(url)
    return IFrame(url, width='100%', height=450)

def fetch_taxon_info(taxon):
    gbif_source_id=11
    ncbi_source_id=4

    url = f"http://resolver.globalnames.org/name_resolvers.json?data_source_ids={gbif_source_id}|{ncbi_source_id}&resolve_once=false&names={taxon}"    
    ids = {'gbif': None, 'ncbi': None}
    classifications = {'gbif': None, 'ncbi': None}
    
    print(url)

    result = requests.get(url)
    data = result.json()['data'][0]

    if data['is_known_name'] == True: 
        results = data['results']

        for value in results:
            if(value['data_source_id'] == gbif_source_id and ids['gbif'] == None):
                ids['gbif'] = value['taxon_id']
                classifications['gbif'] = {'names': value['classification_path'], 'ranks': value['classification_path_ranks']}
            elif(value['data_source_id'] == ncbi_source_id  and ids['ncbi'] == None):
                ids['ncbi'] = value['taxon_id']
                classifications['ncbi'] = {'names': value['classification_path'], 'ranks': value['classification_path_ranks']}

    return {'ids': ids, 'classifications': classifications }

def display_classification(classification):
    if classification:
        names = classification['names'].split('|')
        ranks = classification['ranks'].split('|')
        for index, rank in enumerate(ranks):
            if names[index] == '' :
                continue
            print(f"{rank if rank else 'no rank'}: {names[index]}")
    else:
        print('Taxa not found')
              
def fetch_common_names(taxon):
    url = f"https://api.inaturalist.org/v1/taxa?q={taxon}"
                  
    result = requests.get(url)
    data = result.json()['results']  

    if len(data) > 0:                 
        if 'preferred_common_name' in data[0]:
            return data[0]['preferred_common_name']
        else:
            return 'Common name not found'
    else: 
        return 'Taxa not found'
                  
def display_globi(taxon):
    url = f"https://api.globalbioticinteractions.org/interaction?sourceTaxon={taxon}&type=json.v2"
    result = requests.get(url)
    data = result.json()
                  
    for record in data:
        interaction = re.sub('([A-Z]{1})', r' \1', record['interaction_type']).lower()
        print(f"{record['source_taxon_name']} {interaction} {record['target_taxon_name']}")
    

In [ ]:
info = fetch_taxon_info(TAXON)
ids = info['ids']
classifications = info['classifications']

In [ ]:
print('Classifications')
print()
print('NCBI classification')
print()
display_classification(classifications['ncbi'])
print()
print('GBIF classification')
print()
display_classification(classifications['gbif'])

In [ ]:
name = fetch_common_names(TAXON)

print('Common names')
print()
print(name)


In [ ]:
url = f"https://en.wikipedia.org/wiki/{TAXON}"
show_site(url)

In [ ]:
id = ids['gbif']
url = f"https://www.gbif.org/species/{id}"
show_site(url)

In [ ]:
id = ids['ncbi']
url = f"http://lifemap-ncbi.univ-lyon1.fr/?tid={id}"
show_site(url)

In [ ]:
id = ids['ncbi']
url = f"https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Info&id={id}"
show_site(url)

In [ ]:
display_globi(TAXON)